In [ ]:
import tensorflow as tf
from models.base_model import BaseTFModel
from tensorflow.keras import layers

class NN(BaseTFModel):
    def build_model(self):
        # 1) Coge solo los parámetros de la arquitectura
        mp = self.model_params

        seq_len   = int(mp.get('seq_len',    4096))
        n_classes = int(mp.get('output_size', mp.get('n_classes')))  # o sacar de mp

        # 2) Define tu Input con seq_len
        inp = layers.Input(shape=(seq_len, 2), name='IQ_input')

        # 3) Bloque Inception-1D
        def inception_block(x, f):
            c1 = layers.Conv1D(f, 3, padding='same', activation='relu')(x)
            c2 = layers.Conv1D(f, 5, padding='same', activation='relu')(x)
            c3 = layers.Conv1D(f, 7, padding='same', activation='relu')(x)
            return layers.Concatenate()([c1, c2, c3])

        x = inception_block(inp, mp.get('inception_filters', 32))
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling1D(mp.get('pool_size', 4))(x)

        # 4) Bloques residuales
        def res_block(x, f):
            y = layers.Conv1D(f, 3, padding='same', activation='relu')(x)
            y = layers.BatchNormalization()(y)
            y = layers.Conv1D(f, 3, padding='same')(y)
            y = layers.BatchNormalization()(y)
            if x.shape[-1] != f:
                x = layers.Conv1D(f, 1, padding='same')(x)
            return layers.Activation('relu')(layers.Add()([x, y]))

        for f in mp.get('res_filters', [64, 128]):
            x = res_block(x, f)
            x = layers.MaxPooling1D(mp.get('pool_size', 4))(x)

        # 5) GRU + atención
        x = layers.Bidirectional(
                layers.GRU(mp.get('gru_units', 128), return_sequences=True)
            )(x)
        attn = layers.Dense(1, activation='softmax')(x)   # softmax sobre time-axis
        x = layers.Multiply()([x, attn])
        x = layers.GlobalAveragePooling1D()(x)

        # 6) Clasificador
        x = layers.Dense(mp.get('dense_units', 128), activation='relu')(x)
        x = layers.Dropout(mp.get('dropout', 0.5))(x)
        out = layers.Dense(n_classes, activation='softmax')(x)

        return tf.keras.Model(inputs=inp, outputs=out)
